<a href="https://colab.research.google.com/github/jasonthewhale/2.4.bet-and-wager/blob/main/kaggle_comp_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import numpy as np
from torch.utils import data
from d2l import torch as d2l

In [ ]:
data_path = '/content/drive/MyDrive/kaggle_house_price/'
test_data = pd.read_csv(data_path + 'test.csv')
train_data = pd.read_csv(data_path + 'train.csv')

In [ ]:
train_data_ = train_data.loc[:,train_data.columns != 'Sold Price']
train_data_ = train_data_.loc[:,train_data_.columns != 'Summary']
test_data_ = test_data.loc[:,test_data.columns != 'Summary']

all_features = pd.concat((train_data_.iloc[:, 1:], test_data_.iloc[:, 1:]))

In [ ]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
object_features = all_features.dtypes[all_features.dtypes == 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
        lambda x: (x - x.mean()) / (x.std())
    )
# 在标准化数据之后，所有数据都意味着消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)


In [ ]:
all_features.head()

,Address,Type,Year built,Heating,Cooling,Parking,Lot,Bedrooms,Bathrooms,Full bathrooms,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,540 Pine Ln,SingleFamily,0.048505,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",-0.018489,"Ground Floor Bedroom, Master Bedroom on Ground...",-2.058002,0.000000,...,"Garage, Garage - Attached, Covered",0.194780,0.305277,2019-10-24,1.407144,NaN,0.000000,Los Altos,0.411159,CA
1,1727 W 67th St,SingleFamily,-0.284656,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",-0.018287,3,-0.336769,-0.106424,...,"Detached Carport, Garage",-0.175090,-0.188125,2019-10-16,-0.282410,2019-08-30,-0.365086,Los Angeles,-1.492275,CA
2,28093 Pine Ave,SingleFamily,-0.036722,Forced air,NaN,0 spaces,-0.018031,2,0.523848,-1.167570,...,NaN,-0.616598,-0.639260,2019-08-25,-0.441107,NaN,0.000000,Strawberry,1.059045,CA
3,10750 Braddock Dr,SingleFamily,-0.121949,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",0.000000,5,0.523848,0.954722,...,"Detached Carport, Driveway, Garage - Two Door",1.056241,0.945288,2019-10-24,0.347781,2016-08-30,0.795461,Culver City,-1.404645,CA
4,7415 O Donovan Rd,VacantLand,0.000000,NaN,NaN,0 spaces,0.000000,NaN,0.000000,0.000000,...,NaN,0.000000,0.000000,2019-06-07,0.209783,2016-06-27,0.201324,Creston,0.128637,CA


In [ ]:
main_obj = ['Type', 'Bedrooms']

main_obj = ['Type', 'Cooling', 'Bedrooms', 'Region', 'Cooling features', 'City', 'State']

obj_to_drop = ['Heating', 'Parking', 'Elementary School',
               'Middle School', 'High School', 'Flooring','Heating features',
               'Appliances included','Laundry features',
               'Parking features', 'Listed On', 'Last Sold On', 'Address']

all_features = all_features.drop(obj_to_drop, axis=1)

In [ ]:
all_features.head()

,Type,Year built,Cooling,Lot,Bedrooms,Bathrooms,Full bathrooms,Total interior livable area,Total spaces,Garage spaces,...,High School Score,High School Distance,Cooling features,Tax assessed value,Annual tax amount,Listed Price,Last Sold Price,City,Zip,State
0,SingleFamily,0.048505,"Multi-Zone, Central AC, Whole House / Attic Fan",-0.018489,"Ground Floor Bedroom, Master Bedroom on Ground...",-2.058002,0.000000,-0.006552,-0.135519,-0.123623,...,0.929329,-0.318177,Central,0.194780,0.305277,1.407144,0.000000,Los Altos,0.411159,CA
1,SingleFamily,-0.284656,"Wall/Window Unit(s), Evaporative Cooling, See ...",-0.018287,3,-0.336769,-0.106424,-0.005198,-0.062731,-0.050507,...,-2.101186,-0.318177,"Wall/Window Unit(s), Evaporative Cooling, See ...",-0.175090,-0.188125,-0.282410,-0.365086,Los Angeles,-1.492275,CA
2,SingleFamily,-0.036722,NaN,-0.018031,2,0.523848,-1.167570,-0.004763,-0.135519,-0.123623,...,0.000000,2.350838,NaN,-0.616598,-0.639260,-0.441107,0.000000,Strawberry,1.059045,CA
3,SingleFamily,-0.121949,Central Air,0.000000,5,0.523848,0.954722,-0.002493,-0.135519,-0.123623,...,0.929329,-0.651804,Central Air,1.056241,0.945288,0.347781,0.795461,Culver City,-1.404645,CA
4,VacantLand,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.080842,2.502486,NaN,0.000000,0.000000,0.209783,0.201324,Creston,0.128637,CA


In [ ]:
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features = all_features.astype(float)

In [ ]:
# all_features = all_features.apply(pd.to_numeric, errors='coerce')
# all_features.fillna(0, inplace=True)
# bool_cols = all_features.select_dtypes(include=['bool']).columns
# all_features[bool_cols] = all_features[bool_cols].astype(float)

In [ ]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values,
                              dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values,
                             dtype=torch.float32)
train_labels = torch.tensor(train_data['Sold Price'].iloc[all_features[:n_train].index].values.reshape(-1, 1),
                            dtype=torch.float32)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
loss = nn.MSELoss()
in_features = train_features.shape[1]
print(in_features)

def get_net():
    net = nn.Sequential(nn.Linear(in_features, 256),
                        nn.ReLU(),
                        nn.Linear(256, 64),
                        nn.ReLU(),
                        nn.Linear(64, 1))
    return net

4361


In [ ]:
def log_rmse(net, features, labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()

def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    net = net.to(device)
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    train_features = train_features.to(device)
    train_labels = train_labels.to(device)
    if test_labels is not None:
        test_features = test_features.to(device)
        test_labels = test_labels.to(device)
    # 这里使用的是Adam优化算法
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate,
                                 weight_decay=weight_decay)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

In [ ]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay,
           batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        d2l.plot(list(range(1, num_epochs + 1)), [train_ls, valid_ls],
                     xlabel='epoch', ylabel='rmse', xlim=[1, num_epochs],
                     legend=['train', 'valid'], yscale='log')
        print(f'fold {i + 1}, train log rmse {float(train_ls[-1]):f}, '
              f'valid log rmse {float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

In [ ]:
k, num_epochs, lr, weight_decay, batch_size = 5, 500, 0.02, 0.001, 128
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-cross vali: avg traing log rmse: {float(train_l):f}, '
      f'avg vali log rmse: {float(valid_l):f}')

fold 1, train log rmse 0.296117, valid log rmse 1.022726


In [ ]:
def train_and_pred(train_features, test_feature, train_labels, test_data,
                   num_epochs, lr, weight_decay, batch_size):
    net = get_net()
    train_ls, _ = train(net, train_features, train_labels, None, None,
                        num_epochs, lr, weight_decay, batch_size)
    d2l.plot(np.arange(1, num_epochs + 1), [train_ls], xlabel='epoch',
             ylabel='log rmse', xlim=[1, num_epochs], yscale='log')
    print(f'train log rmse {float(train_ls[-1]):f}')
    # 将网络应用于测试集。
    print(test_features.shape)
    preds = net(test_features.to(device)).detach().cpu().numpy()
    # 将其重新格式化以导出到Kaggle
    test_data['Sold Price'] = pd.Series(preds.reshape(1, -1)[0])
    submission = pd.concat([test_data['Id'], test_data['Sold Price']], axis=1)
    submission.to_csv('submission.csv', index=False)
    return submission

In [ ]:
submission = train_and_pred(train_features, test_features, train_labels, test_data,
               num_epochs, lr, weight_decay, batch_size)